In [1]:
from __future__ import unicode_literals  # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
from google.cloud import vision
from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup

## 라프텔 썸네일 크롤링
- 전체 애니메이션 썸네일 긁어오기

In [58]:
def crawling_laftel_thumbnail():
    laftel_thumbnail_data = [] 
    total_img_count = 0
    error_img_count = 0

    domain_url = 'https://laftel.net'
    main_url = 'https://laftel.net/finder'

    chrome_options = Options()
    chrome_options.add_argument("--window-size=1600,1080")

    # if headless == True:
    #     chrome_options.add_argument('headless')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(main_url)
    
    driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div[1]/div/div/div[1]/div/div[2]/section/div[1]').click()
    
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(SCROLL_PAUSE_TIME)

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        last_height = new_height
        
    if not os.path.exists('./{}'.format('total_img')):
        os.mkdir('./{}'.format('total_img'))
        
    thumbnail_ele_list = driver.find_elements_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div/img')
    
    for each_ele in thumbnail_ele_list:
        error_img = False
        img_url = each_ele.get_attribute('src')
        title = each_ele.get_attribute('alt').split(' ')[0]
        title = "&".join(title.split('/'))
        
        if img_url == 'https://image.laftel.net/':
            error_img = True
    
        laftel_thumbnail_data.append({'title': title, 'img_url': img_url, "error_img": error_img})
        
        if img_url == 'https://image.laftel.net/':
            error_img_count += 1
            continue
        
        urllib.request.urlretrieve(img_url, './{}/{}.png'.format('total_img', title) + '_' + 'thumbnail')
        total_img_count += 1
        
    laftel_thumbnail_df = pd.DataFrame(laftel_thumbnail_data, columns=['title', 'img_url'])
    
    return laftel_thumbnail_df
        
    

In [40]:
"&".join(title.split('/'))

'Fate&stay'

In [41]:
"&".join('adffa'.split('/'))

'adffa'

In [43]:
each_ele

<selenium.webdriver.remote.webelement.WebElement (session="91c153243575c7f149b7327561bcb9c7", element="aeae23b2-d54a-4bfb-923c-bd0dc10bf9be")>

In [44]:
each_ele.get_attribute('src')

'https://image.laftel.net/'

In [46]:
len(thumbnail_ele_list)

5737

In [48]:
thumbnail_url_list = [each_ele.get_attribute('src') for each_ele in thumbnail_ele_list]

In [51]:
for each_thumbnail in thumbnail_url_list:
    if each_thumbnail == 'https://image.laftel.net/':
        print(each_thumbnail)

https://image.laftel.net/
https://image.laftel.net/
https://image.laftel.net/
https://image.laftel.net/
https://image.laftel.net/
https://image.laftel.net/
https://image.laftel.net/
https://image.laftel.net/
https://image.laftel.net/
https://image.laftel.net/


In [52]:
laftel_thumbnail_df = pd.DataFrame(thumbnail_url_list, columns=['url'])

In [53]:
laftel_thumbnail_df

,url
0,https://image.laftel.net/items/thumbs/big/f320...
1,https://image.laftel.net/items/thumbs/big/167c...
2,https://image.laftel.net/items/thumbs/big/9ec0...
3,https://image.laftel.net/items/thumbs/big/f3d8...
4,https://image.laftel.net/items/thumbs/big/4b63...
...,...
5732,https://image.laftel.net/items/thumbs/big/1148...
5733,https://image.laftel.net/items/thumbs/big/2573...
5734,https://image.laftel.net/items/thumbs/big/622a...
5735,https://image.laftel.net/items/thumbs/big/73ce...


In [55]:
laftel_thumbnail_df.to_csv('laftel_thumbnail_url.csv')